In [4]:
import arcpy


#paths for input FCs
featureclass1_edw = arcpy.GetParameterAsText(0)
featureclass2_nrm = arcpy.GetParameterAsText(1)
workspace = arcpy.GetParameterAsText(3)
output_diff = f"{workspace}\\NotInEDWInput"
output_diff1 = f"{workspace}\\NotInNRMInput"

arcpy.env.overwriteOutput = True

#make sure inputs have features
featurecountinput1 = int(arcpy.GetCount_management(featureclass1_edw).getOutput(0))
if featurecountinput1 == 0:
    arcpy.AddError("{0} had no features.".format(featureclass1_edw))
else:
    arcpy.AddMessage("{0} has {1} features.".format(featureclass1_edw, featurecountinput1))

featurecountinput2 = int(arcpy.GetCount_management(featureclass2_nrm).getOutput(0))
if featurecountinput2 == 0:
    arcpy.AddError("{0} had no features.".format(featureclass2_nrm))
else:
    arcpy.AddMessage("{0} has {1} features.".format(featureclass2_nrm, featurecountinput2))

#export the two source data featurclasses to the file geodatabase
try:
    #ensure FGDB exists
    if not arcpy.Exists(workspace):
        arcpy.AddError(f"The output file geodatabase does not exist: {workspace}")
        raise Exception("Output FGDB does not exist.")
#export the first FC
    #get name of first featureclass
    outputedwdata_name = 'EDWSourceData'
    outputedwdata = f"{workspace}\\{outputedwdata_name}"
    arcpy.CopyFeatures_management(featureclass1_edw, outputedwdata)
    arcpy.AddMessage(f"Exported '{outputedwdata_name}' to {workspace}.")

    #get name of second featureclass
    outputnrmdata_name = 'NRMSourceData'
    outputnrmdata = f"{workspace}\\{outputnrmdata_name}"
    arcpy.CopyFeatures_management(featureclass2_nrm, outputnrmdata)
    arcpy.AddMessage(f"Exported '{outputnrmdata_name}' to {workspace}.")

except Exception as e:
    arcpy.AddError(f"An error occured: {e}")
    raise

#common field compare
common_field = "SUID"

#dictionary of values from featureclass2_nrm
fc2_values = {row[0] for row in arcpy.da.SearchCursor(outputnrmdata, [common_field])}

#use search cursor to find diffrences and insert them into a new featureclass
with arcpy.da.SearchCursor(outputedwdata, [common_field, "SHAPE@"]) as cursor:
    differences = [row for row in cursor if row[0] not in fc2_values]

if differences:
    #create empty fc in fgdb
    spatial_reference = arcpy.Describe(outputedwdata).spatialReference
    arcpy.management.CreateFeatureclass(workspace, "NotInEDWInput", "POLYGON", featureclass1_edw)

    #insert diffrences in a new fc
    with arcpy.da.InsertCursor(output_diff, [common_field, "SHAPE@"]) as insert_cursor:
        for diff in differences:
            insert_cursor.insertRow(diff)

    arcpy.AddMessage(f"Diffrences found in EDW data and saved to {output_diff}")
else:
    arcpy.AddMessage("No diffrences found in EDW dataset.")

#dictionary of values from featureclass1_edw
fc2_values = {row[0] for row in arcpy.da.SearchCursor(outputedwdata, [common_field])}

#use search cursor to find diffrences and insert them into a new featureclass
with arcpy.da.SearchCursor(outputnrmdata, [common_field, "SHAPE@"]) as cursor:
    differences = [row for row in cursor if row[0] not in fc2_values]

if differences:
    #create empty fc in fgdb
    spatial_reference = arcpy.Describe(outputnrmdata).spatialReference
    arcpy.management.CreateFeatureclass(workspace, "NotInNRMInput", "POLYGON", featureclass2_nrm)

    #insert diffrences in a new fc
    with arcpy.da.InsertCursor(output_diff1, [common_field, "SHAPE@"]) as insert_cursor:
        for diff in differences:
            insert_cursor.insertRow(diff)

    arcpy.AddMessage(f"Diffrences found in NRM data and saved to {output_diff1}")
else:
    arcpy.AddMessage("No diffrences found in NRM dataset.")
        



Diffrences found in EDW data and saved to T:\FS\NFS\R09\Program\6800InformationManagement\GIS\Workspace\jerwin\Tools\DataCompareTool\DCTesting2.gdb\NotInEDWInput
No diffrences found in NRM dataset.
